In [1]:
!gdown --id 1_--Ku9Ejld0BsovvT0SZ5AReZ1_93NFz

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1_--Ku9Ejld0BsovvT0SZ5AReZ1_93NFz
To: /content/DEAP.zip
100% 2.92G/2.92G [00:31<00:00, 92.9MB/s]


In [2]:
!unzip DEAP.zip

Archive:  DEAP.zip
   creating: DEAP/
  inflating: DEAP/s01.dat            
  inflating: DEAP/s02.dat            
  inflating: DEAP/s03.dat            
  inflating: DEAP/s04.dat            
  inflating: DEAP/s05.dat            
  inflating: DEAP/s06.dat            
  inflating: DEAP/s07.dat            
  inflating: DEAP/s08.dat            
  inflating: DEAP/s09.dat            
  inflating: DEAP/s10.dat            
  inflating: DEAP/s11.dat            
  inflating: DEAP/s12.dat            
  inflating: DEAP/s13.dat            
  inflating: DEAP/s14.dat            
  inflating: DEAP/s15.dat            
  inflating: DEAP/s16.dat            
  inflating: DEAP/s17.dat            
  inflating: DEAP/s18.dat            
  inflating: DEAP/s19.dat            
  inflating: DEAP/s20.dat            
  inflating: DEAP/s21.dat            
  inflating: DEAP/s22.dat            
  inflating: DEAP/s23.dat            
  inflating: DEAP/s24.dat            
  inflating: DEAP/s25.dat            
  inflating:

In [1]:
import glob
import os
filelist = glob.glob('DEAP/*.dat')
print(filelist)
subjectnames = [fr[5:8] for fr in filelist]
print(subjectnames)

['DEAP/s18.dat', 'DEAP/s26.dat', 'DEAP/s01.dat', 'DEAP/s12.dat', 'DEAP/s29.dat', 'DEAP/s04.dat', 'DEAP/s06.dat', 'DEAP/s31.dat', 'DEAP/s17.dat', 'DEAP/s13.dat', 'DEAP/s30.dat', 'DEAP/s21.dat', 'DEAP/s27.dat', 'DEAP/s05.dat', 'DEAP/s24.dat', 'DEAP/s11.dat', 'DEAP/s10.dat', 'DEAP/s25.dat', 'DEAP/s14.dat', 'DEAP/s02.dat', 'DEAP/s15.dat', 'DEAP/s08.dat', 'DEAP/s23.dat', 'DEAP/s16.dat', 'DEAP/s20.dat', 'DEAP/s22.dat', 'DEAP/s09.dat', 'DEAP/s32.dat', 'DEAP/s19.dat', 'DEAP/s07.dat', 'DEAP/s28.dat', 'DEAP/s03.dat']
['s18', 's26', 's01', 's12', 's29', 's04', 's06', 's31', 's17', 's13', 's30', 's21', 's27', 's05', 's24', 's11', 's10', 's25', 's14', 's02', 's15', 's08', 's23', 's16', 's20', 's22', 's09', 's32', 's19', 's07', 's28', 's03']


In [2]:
import pickle
data = {}
for sname in subjectnames:
    dname = "DEAP/"+sname+".dat"
    f = open(dname, 'rb')
    x = pickle.load(f, encoding='latin1')
    data[sname] = x
print(data.keys())

dict_keys(['s18', 's26', 's01', 's12', 's29', 's04', 's06', 's31', 's17', 's13', 's30', 's21', 's27', 's05', 's24', 's11', 's10', 's25', 's14', 's02', 's15', 's08', 's23', 's16', 's20', 's22', 's09', 's32', 's19', 's07', 's28', 's03'])


In [3]:
import numpy as np
data_c = {}
for k,v in data.items():
    y = data[k]['data'][:,36,:]
    ym = np.mean(y,axis=-1).reshape(40,1)
    ystd = np.std(y,axis=-1).reshape(40,1)
    z = (y-ym)/ystd
    data_c[k] = [z,data[k]['labels']]

In [6]:
#!git clone https://github.com/HealthSciTech/pyEDA.git

Cloning into 'pyEDA'...
remote: Enumerating objects: 290, done.
remote: Counting objects: 100% (290/290), done.
remote: Compressing objects: 100% (198/198), done.
remote: Total 290 (delta 151), reused 203 (delta 87), pack-reused 0
Receiving objects: 100% (290/290), 11.30 MiB | 26.55 MiB/s, done.
Resolving deltas: 100% (151/151), done.


In [4]:
#!pip install cvxopt

In [5]:
from pyEDA.main import *
from scipy.stats import skew
from scipy.stats import kurtosis
import os,sys


data_c1 = {}
for k,v in data_c.items():
    y = data_c[k][0]
    data_c1[k] = y


In [6]:
data_c1p = {}
BLOCK_SIZE=512
BLOCK_STRIDE=64
for k,v in data_c1.items():
    datablocki = []
    for i in range(40):
        sh = (v[i].size - BLOCK_SIZE + 1, BLOCK_SIZE)
        st = v[i].strides * 2
        view = np.lib.stride_tricks.as_strided(v[i], strides = st, shape = sh)[0::BLOCK_STRIDE]
        data0_0 = view.copy()
        data0_0 = data0_0[np.newaxis,:,:]
        data0_0 = data0_0.reshape(-1,1,512)
        datablocki.append(torch.tensor(data0_0))
    data_c1p[k] = datablocki
print(data_c1p['s01'][0].shape)

torch.Size([119, 1, 512])


In [7]:
data_c2 = {}
for k,v in data_c.items():
    y = data_c[k][1]
    x_label = np.zeros((y.shape[0]),dtype='int8')
    for i in range(y.shape[0]):
        if (y[i][0] > 5 and y[i][1] > 5):
            x_label[i] = 3
        elif (y[i][0] <= 5 and y[i][1] > 5):
            x_label[i] = 3
        elif (y[i][0] > 5 and y[i][1] <= 5):
            x_label[i] = 0
        elif (y[i][0] <= 5 and y[i][1] <= 5):
            x_label[i] = 0
    x_l = np.zeros((x_label.size, x_label.max()+1))
    x_l[np.arange(x_label.size), x_label] = 1
    #
    #print(x_l.shape)
    x_l = x_l.reshape(-1,1,4)
    x_l = np.repeat(x_l, 119, axis=1)
    #print(x_l.shape)
    x_l = torch.tensor(x_l)
    data_c2[k] = x_l

In [8]:
print(data_c2['s01'].shape)

torch.Size([40, 119, 4])


In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.c1=nn.Conv1d(1, 24, 5,stride=3)
        self.norm1 = nn.BatchNorm1d(24)
        self.c2=nn.Conv1d(24, 16, 3,stride=2)
        self.norm2 = nn.BatchNorm1d(16)
        self.c3=nn.Conv1d(16, 8, 3,stride=2)
        self.norm3 = nn.BatchNorm1d(8)
        self.ft = nn.Flatten()
        self.n1 = nn.Linear(328,20)
        self.d = nn.Dropout()
        self.n2 = nn.Linear(20,4)


    def forward(self, x):
        x = F.relu(self.norm1(self.c1(x)))
        x = F.relu(self.norm2(self.c2(x)))
        x = F.relu(self.norm3(self.c3(x)))

        x = self.ft(x)
        x = self.d(F.relu(self.n1(x)))
        x = F.softmax(self.n2(x),dim=-1)
        return x


criterion = nn.CrossEntropyLoss()


In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix,classification_report
from sklearn import preprocessing
#subjectnames
file_list_num = np.arange(len(subjectnames))
kf = KFold(n_splits=8)
#file_list_num
for i, (train_index, test_index) in enumerate(kf.split(file_list_num)):
    print(f"Fold {i}:")
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")
    net = Net()
    net.to(device)
    optimizer = optim.Adam(net.parameters(), lr=0.00001,betas=(0.9,0.999))
    epochs = 500
    for epoch in range(epochs):
        for tr in train_index:
            v = data_c1p[subjectnames[tr]]
            l = data_c2[subjectnames[tr]]
            net.train()
            for i in range(len(v)):
                optimizer.zero_grad()
                outputs = net(v[i].to(device, dtype=torch.float))
                loss = criterion(outputs, l[i].to(device))
                loss.backward()
                optimizer.step()
            #print(f'Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}')
    val_loss = []
    expectedoutput = []
    actualoutput = []
    for tr in test_index:
        net.eval()
        v = data_c1p[subjectnames[tr]]
        l = data_c2[subjectnames[tr]]
        with torch.no_grad():
            for i in range(len(v)):
                outputs = net(v[i].to(device, dtype=torch.float))
                loss = criterion(outputs, l[i].to(device))
                val_loss.append(loss)
                output1 = torch.argmax(l[i],dim=1)
                aoutput1 = torch.argmax(outputs.cpu(),dim=1)
                expectedoutput.append(torch.mode(output1).values.numpy())
                actualoutput.append(torch.mode(aoutput1).values.numpy())
    val_loss_mean = torch.stack(val_loss).mean()
    print(classification_report(expectedoutput,actualoutput))
    print(f'Validation Loss for {subjectnames[tr]} = {val_loss_mean}')

Fold 0:
  Train: index=[ 4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27
 28 29 30 31]
  Test:  index=[0 1 2 3]
              precision    recall  f1-score   support

           0       1.00      0.03      0.06        63
           3       0.61      1.00      0.76        97

    accuracy                           0.62       160
   macro avg       0.81      0.52      0.41       160
weighted avg       0.77      0.62      0.49       160

Validation Loss for s12 = 1.1454256795565871
Fold 1:
  Train: index=[ 0  1  2  3  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27
 28 29 30 31]
  Test:  index=[4 5 6 7]
